<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-предподоготовка-данных" data-toc-modified-id="Загрузка-и-предподоготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и предподоготовка данных</a></span><ul class="toc-item"><li><span><a href="#Загрузка-файлов" data-toc-modified-id="Загрузка-файлов-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка файлов</a></span></li><li><span><a href="#Предподготовка-данных" data-toc-modified-id="Предподготовка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Предподготовка данных</a></span></li></ul></li><li><span><a href="#Анализ-данных" data-toc-modified-id="Анализ-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Анализ данных</a></span></li><li><span><a href="#Анализ-воронки-событий" data-toc-modified-id="Анализ-воронки-событий-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Анализ воронки событий</a></span><ul class="toc-item"><li><span><a href="#События" data-toc-modified-id="События-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>События</a></span></li><li><span><a href="#События-по-числу-пользователей" data-toc-modified-id="События-по-числу-пользователей-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>События по числу пользователей</a></span></li><li><span><a href="#Доля-пользователей,-переходящая-на-следующий-шаг-воронки" data-toc-modified-id="Доля-пользователей,-переходящая-на-следующий-шаг-воронки-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Доля пользователей, переходящая на следующий шаг воронки</a></span></li><li><span><a href="#Пользователи,-проходящие-от-первого-события-до-оплаты" data-toc-modified-id="Пользователи,-проходящие-от-первого-события-до-оплаты-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Пользователи, проходящие от первого события до оплаты</a></span></li></ul></li><li><span><a href="#Результаты-эксперимента" data-toc-modified-id="Результаты-эксперимента-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Результаты эксперимента</a></span><ul class="toc-item"><li><span><a href="#Количество-пользователей-в-каждой-экспериментальной-группе" data-toc-modified-id="Количество-пользователей-в-каждой-экспериментальной-группе-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Количество пользователей в каждой экспериментальной группе</a></span></li><li><span><a href="#Статистическая-значимость-различий-между-выборками-246-и-247--для-А/А-эксперимента" data-toc-modified-id="Статистическая-значимость-различий-между-выборками-246-и-247--для-А/А-эксперимента-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Статистическая значимость различий между выборками 246 и 247  для А/А-эксперимента</a></span></li><li><span><a href="#A/B-тест" data-toc-modified-id="A/B-тест-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>A/B-тест</a></span></li></ul></li></ul></div>

# Анализ поведения пользователей в мобильном приложении

**Цель проекта:**
Разобраться, как ведут себя пользователи вашего мобильного приложения. 

Для достижения этой цели, необходимо решить следующие **задачи:**
- Изучить воронку продаж. Узнать, как пользователи доходят до покупки. Сколько пользователей доходит до покупки. Сколько пользователей доходит до покупки, а сколько — «застревает» на предыдущих шагах? На каких именно?
- Исследовать результаты A/A/B-эксперимента. Пользователей разбили на 3 группы: 2 контрольные со старыми шрифтами и одну экспериментальную — с новыми. 

**План проекта:**
1. Загрузка и предподоготовка данных
2. Анализ данных
3. Анализ воронки событий
4. Изучение результатов эксперимента: 
- A/A-тест;
- A1/B-тест;
- A2/B-тест;
- A1+2/B-тест

## Загрузка и предподоготовка данных

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import matplotlib
import scipy.stats as stats
import math as mth
from scipy import stats as st
import seaborn as sns  
import plotly.express as px

### Загрузка файлов

In [5]:
try:
    logs = pd.read_csv('/datasets/logs_exp.csv', sep='\t')
except:
    logs = pd.read_csv('logs_exp.csv', sep='\t')

In [ ]:
logs.info()

In [ ]:
logs.head()

По данным видно следующее: нет пропущенных данных, один столбец с типом данных `object`, еще три `int64`, всего в датафрейме 244126 строк.

Перед анализом необходимо провести следующую предобработку:
- изменение названия столбцов;
- проверить `EventName` на неявные дубликаты;
- привести столбец `EventTimestamp` к типу данных `datetime`;
- проверить на дубликаты.

### Предподготовка данных

In [ ]:
# изменяем названия столбцов
logs.columns = ['event', 'user_id', 'dt', 'exp_id']
logs.info()

Посмотрим, какие события имеются в нашем датафрейме и нет ли неявных дубликатов

In [ ]:
logs['event'].unique()

С этим столбцов все в порядке, ничего не будем менять

In [ ]:
# изменим в столбце dt тип данных timestamp на datetime
logs['dt'] = pd.to_datetime(logs['dt'], unit='s')
logs.info()

In [ ]:
# добавим столбец date
logs['date'] = logs['dt'].dt.date
logs.head()

Проверим датафрейм на дубликаты и удалим их при наличии

In [ ]:
duplicates = logs.duplicated().sum()
duplicates

In [ ]:
print('Дубликаты занимают', round(duplicates/len(logs['event']), 5)*100, '% от общего числа данных')

In [ ]:
logs = logs.drop_duplicates()
logs.duplicated().sum()

**Вывод по предобработке данных:**
   
   Были изменены названия столбцов, изменен тип данных для столбца с датой и временем, а также добавлен столбец с датой, удалено 413 дубликатов. Теперь данные готовы к анализу.

## Анализ данных

Определим количество событий в логе

In [ ]:
logs['event'].nunique()

In [ ]:
logs['event'].value_counts()

Всего в логе пять вариантов событий, перечислим их по убыванию: показ главного экрана, показ экрана с предложением, показ экрана с корзиной, оплата прошла успешно, обучение. Как мы видим, около трети пользователей, зашедших в приложение переходят на экран с предложением, из них не все переходят в корзину и при этом не все перешедшие в корзину оплачивают покупку.

Теперь опередим количество пользователей в логе

In [ ]:
print('Всего в логе', logs['user_id'].nunique(), 'пользователь')

Определим, сколько в среднем событий приходится на пользователя

In [ ]:
print('В среднем на одного пользователя', round(len(logs['event'])/logs['user_id'].nunique()), 'события')

In [ ]:
# вычисляем количество событий для каждого уникального пользователя
event_per_user = logs.groupby('user_id').agg({'event':'count'}).reset_index()
event_per_user

Построим диаграмму размаха для определения аномальных пользователей, совершивших слишком большое количество событий.

In [ ]:
plt.figure(figsize=(10, 6))
plt.title('Диаграмма размаха количества событий')
sns.boxplot(y='event', data=event_per_user).set(xlabel='', ylabel='Количество событий')
plt.xticks(rotation= 45)
plt.show()


In [ ]:
# определим 99 перцентиль методом np.percentile()
np.percentile(event_per_user['event'],[99])

График показывает много аномалий, а расчет 99 перцентиля показывает, что более 200 событий, совершенными одним пользователем считаются аномальными, поэтому от них необходимо избавиться.

In [ ]:
# получим список аномальных пользователей
user_id_anomaly = list(event_per_user.query('event > 200')['user_id'])
print('Всего', len(user_id_anomaly), 'аномальных пользователей')

In [ ]:
# отфильтруем данные с аномальными пользователями
logs_without_anomaly = logs.query('user_id not in @user_id_anomaly') 
# определим, много ли мы удалили
shape_logs = logs.shape # получаем количество строк и столбцов в первоначальном датафрейме
shape_logs_without_anomaly = logs_without_anomaly.shape # получаем количество строк и столбцов в отфильтрованном датафрейме

shape_logs_without_anomaly[0] / shape_logs[0] # интересуют только строки

После фильрации осталось 86% данных

In [ ]:
print('После удаления аномалий в среднем на одного пользователя',
      round(len(logs_without_anomaly['event'])/logs_without_anomaly['user_id'].nunique()),
      'событий')

In [ ]:
logs_without_anomaly['date'].min(), logs_without_anomaly['date'].max()

Мы располагаем данными за период с 25 июля по 7 августа 2019 года

In [ ]:
logs_without_anomaly['date'].value_counts()

In [ ]:
# улучшение качества графиков
%config InlineBackend.figure_format = 'retina'

In [ ]:
matplotlib.style.use('ggplot') 

In [ ]:
logs_without_anomaly.pivot_table(index='date', columns='exp_id', values='user_id', aggfunc='count')

In [ ]:
logs_without_anomaly.pivot_table(index='date', columns='exp_id', values='user_id', aggfunc='count').plot(kind='bar')
plt.title('События в зависимости от даты')
plt.xlabel('Дата')
plt.ylabel('Количество событий')
plt.show()

Получается, что на самом деле мы располагаем данными с 1 по 8 августа данные, т.к. они наиболее полные, а все данные до этой даты можем отбросить.

In [ ]:
logs_without_anomaly['date'] = pd.to_datetime(logs['date'])
logs_actual = logs_without_anomaly.query('date > "2019-07-31"')
logs_actual['date'] = logs_actual['date'].dt.date
logs_actual.pivot_table(index='date', columns='exp_id', values='user_id', aggfunc='count').plot(kind='bar')
plt.grid(True)
plt.xticks(rotation=90)
plt.title('События в зависимости от даты')
plt.xlabel('Дата')
plt.ylabel('Количество событий')
plt.show()

После фильтрации остались данные с 1 по 7 августа 2019 и в каждой исследуемой группе примерно одинаковое число пользователей.

Проверим, какая доля данных от начального числа осталась для исследования, используя DataFrame.shape

In [ ]:
shape_data = logs.shape # получаем количество строк и столбцов в первоначальном датафрейме
shape_data_opt = logs_actual.shape # получаем количество строк и столбцов в отфильтрованном датафрейме

shape_data_opt[0] / shape_data[0] # интересуют только строки

Получается, после фильтрации осталось 85% данных, поэтому это не вредит на актуальность исследуемых данных. Зато исследование будет более объективным.

In [ ]:
print('Было потеряно', shape_data[0] - shape_data_opt[0], 'событий из', shape_data[0])

In [ ]:
print(
    'Было потеряно', 
    logs['user_id'].nunique() - logs_actual['user_id'].nunique(), 
    'пользователя из', logs['user_id'].nunique()
)

Проверим, остались ли пользователи из всех трёх экспериментальных групп

In [ ]:
logs_actual.groupby('exp_id').agg({'user_id': 'nunique'})

В каждой экспериментальной группе осталось примерно одинакое количество уникальных пользователей в районе 2500.

In [ ]:
users = logs_actual.pivot_table(index='dt', columns='exp_id', values='user_id', aggfunc='first')
users.columns = [246, 247, 248]
users

In [ ]:
dupl = []
for i in range(0, len(users)):
    if users[246][i] == users[247][i] or users[246][i] == users[248][i] or users[247][i] == users[248][i]:
        dupl.append([i])
print(dupl)

In [ ]:
logs_actual.groupby('user_id')['exp_id'].nunique().reset_index().query('exp_id > 1')

Список пустой, значит, нет пересечений в группах, следовательно, каждый пользователь только в одной группе.

**Вывод по анализу данных:**

Всего в логе пять вариантов событий, перечислим их по убыванию: показ главного экрана, показ экрана с предложением, показ экрана с корзиной, оплата прошла успешно, обучение. 

Всего в логе 7551 пользователь, но оказалось среди них много аномалий, а расчет 99 перцентиля показывает, что более 200 событий , совершенными одним пользователем считаются аномальными, поэтому от них избавились, таких пользователей оказалось 76. После удаления данных с аномальными пользователями, осталось 86% данных от исходного количества.

Хотя данные представлены за период с 25 июля по 7 августа 2019 года, но было принято решение отбросить все данные до 1 августа, т.к. они составляют менее 2% от общего числа и количество событий в эти даты очень мало, что может повлиять на объективность результатов в дальнейшем. 

Таким образом было отброшено 37098 событий из 243713 и 93 пользователя из 7551, при этом в каждой из трёх экспериментальных групп осталось равномерное количество уникальных пользователей (2456, 2491, 2511), при этом каждый пользователь находится только в одной экспериментальной группе.

## Анализ воронки событий

### События

Посмотрим, какие события есть в логах, как часто они встречаются. Отсортируйте события по частоте.

In [ ]:

(
    logs_actual.groupby('event')['event'].agg('count').sort_values(ascending=False)
    .plot(kind='pie', labels=None, label='', autopct='%1.0f%%', 
       legend=True, figsize=(7, 7), title='Разбивка по событиям')
)
plt.show()

Всего в логе пять вариантов событий, перечислим их по убыванию: показ главного экрана (55%), показ экрана с предложением (20%), показ экрана с корзиной (14%), оплата прошла успешно(14%), обучение (около 0%).

Событие, связанное с обучением, лишнее. Скорее всего оно не влияет на факт оплаты, т.к. видно на графике, что пользователи пользуются сайтом и без обучения.

In [ ]:
logs_new = logs_actual.query('event != "Tutorial"')

### События по числу пользователей

Посчитаем, сколько пользователей совершали каждое из этих событий. 

In [ ]:
logs_groupby_event = (
    logs_new.groupby('event')
    .agg({'user_id': 'nunique'})
    .sort_values(by='user_id', ascending=False)
)
logs_groupby_event.columns = ['users_cnt']
logs_groupby_event

In [ ]:
logs_groupby_event['part_of_users'] = (
    logs_groupby_event['users_cnt']
    .apply(lambda x: round(x/logs_new['user_id'].nunique()*100, 1))
)
logs_groupby_event

98,5% пользователей посещали главный экран, 61% - переходили на страницу с предложением товаров, 49,6% - переходили в корзину и 47% пользователей успешно приобрели продукты в мобильном приложении. 

Видно, что все события сужаются в воронку продаж, начиная с посещения приложения и заканчивая покупкой, но по пути пользователи отсеиваются и до покупки добираются только половина.

Предположительно последовательность событий в приложении следующая:
- пользователь заходит на сайт,
- просматривает каталог с продуктами,
- добавляет в корзину,
- переходит на страницу оплаты и оплачивает заказ.

Также можно зайти в раздел обучения, но пользователи им практически не пользуются, поэтому это событие не входит в воронку продаж.

### Доля пользователей, переходящая на следующий шаг воронки

По воронке событий посчитаем, какая доля пользователей проходит на следующий шаг воронки (от числа пользователей на предыдущем). 

Посмотрим, все ли пользователи попадают на последующий шаг после предыдущего, для этого воспользуемся методом `pivot_table` и столбцами укажем события

In [ ]:
logs_new_pt = logs_new.pivot_table(index='user_id', 
                         columns='event', 
                         values='dt',
                         aggfunc='min')
logs_new_pt = logs_new_pt[[
    'MainScreenAppear',
    'OffersScreenAppear',
    'CartScreenAppear',
    'PaymentScreenSuccessful'    
]]
logs_new_pt.columns = [
    'MainScreenAppear',
    'OffersScreenAppear',
    'CartScreenAppear',
    'PaymentScreenSuccessful'
]
logs_new_pt.tail(20)

Результат показывает, что пользователи могут попадать на разные экраны не последовательно, а также мы видим, что на каждом этапе пользователи отсеиваются, не доходя до оплаты.

In [ ]:
# Создадим столбец конверсий в таблице logs_groupby_event и рассчитаем их в цикле
# цикл проходит по строкам и рассчитывает конверсию между последующим и предыдущим значениями 'users_cnt'
# а для первого значения, где нет предыдущего указываем 100%
logs_groupby_event['conversion'] = 0
for i in range(0, len(logs_groupby_event['part_of_users'])):
    if i == 0:
        logs_groupby_event['conversion'].iloc[i] = 100 
    else:
        logs_groupby_event['conversion'].iloc[i] = round(
            logs_groupby_event['users_cnt'].iloc[i] / logs_groupby_event['users_cnt'].iloc[i-1], 3
        )*100
logs_groupby_event

**Расчеты показали, что из главного экрана в каталог переходят только 62%, из них 81% переходит в корзину и из тех, кто перешел в корзину 95% пользователей оплачивают заказ. Таким образом, на втором событии "Экран предложений" теряется больше всего пользователей.**

### Пользователи, проходящие от первого события до оплаты

In [ ]:
print('До оплаты доходит', 
      round(logs_groupby_event['users_cnt'].iloc[3]/logs_groupby_event['users_cnt'].iloc[0], 4)*100, 
      '% пользователей')

In [ ]:
# построим график воронки событий (по количеству пользователей) с помощью диаграммы воронки с plotly.express
data = dict(
    number = logs_groupby_event['users_cnt'],
    stage=['MainScreenAppear',
    'OffersScreenAppear',
    'CartScreenAppear',
    'PaymentScreenSuccessful'])
fig = px.funnel(data, x='number', y='stage')
fig.show()


**Вывод по анализу воронки событий:**

Всего в логе пять вариантов событий: 
- показ главного экрана (55%), 
- показ экрана с предложением (20%), 
- показ экрана с корзиной (14%), 
- оплата прошла успешно(14%), 
- обучение (около 0%).


Событие, связанное с обучением, лишнее. Скорее всего оно не влияет на факт оплаты, т.к. видно на графике, что пользователи пользуются сайтом и без обучения.

98,5% пользователей посещали главный экран, 61% - переходили на страницу с предложением товаров, 49,6% - переходили в корзину и 47% пользователей успешно приобрели продукты в мобильном приложении.

Видно, что все события сужаются в воронку продаж, начиная с посещения приложения и заканчивая покупкой, но по пути пользователи отсеиваются и до покупки добираются только половина.

Пользователи могут попадать на разные экраны не последовательно, а также на каждом этапе пользователи отсеиваются, не доходя до оплаты.

Расчеты показали, что из главного экрана в каталог переходят только 62%, из них 81% переходит в корзину и из тех, кто перешел в корзину 95% пользователей оплачивают заказ. Таким образом, на втором событии "Экран предложений" теряется больше всего пользователей.

От первого события до оплаты доходят 47.15% пользователей.

## Результаты эксперимента

### Количество пользователей в каждой экспериментальной группе

In [ ]:
logs_new.head()

In [ ]:
logs_exp_total = logs_new.groupby('exp_id').agg({'user_id': 'nunique'})
logs_exp_total.columns = ['total']
logs_exp_total

В каждой экспериментальной группе примерно одинаковое количество уникальных пользователей в районе 2500.

### Статистическая значимость различий между выборками 246 и 247  для А/А-эксперимента

Сформулируем гипотезы:
- Нулевая: различий в долях пользователей между выборками 246 и 247 нет.
- Альтернативная: различия в долях пользователей есть.

In [ ]:
logs_exp = logs_new.pivot_table(index='exp_id', columns='event', values='user_id', aggfunc='nunique')
logs_exp = logs_exp[[
    'MainScreenAppear',
    'OffersScreenAppear',
    'CartScreenAppear',
    'PaymentScreenSuccessful'    
]]
logs_exp.columns = [
    'MainScreenAppear',
    'OffersScreenAppear',
    'CartScreenAppear',
    'PaymentScreenSuccessful'    
]
logs_exp

In [ ]:
# объединим таблицы logs_exp_total и logs_exp методом merge по индексам

logs_merged = pd.merge(logs_exp, logs_exp_total, left_index=True, right_index=True)
logs_merged.columns = [
    'MainScreenAppear',
    'OffersScreenAppear',
    'CartScreenAppear',
    'PaymentScreenSuccessful',
    'total'
]
logs_merged

In [ ]:
# добавим функцию для проверки гипотезы
def hypothesis_test (successes, trials, alpha):
    # пропорция успехов в первой группе:
    p1 = successes[0]/trials[0]
    # пропорция успехов во второй группе:
    p2 = successes[1]/trials[1]
    # пропорция успехов в комбинированном датасете:
    p_combined = (successes[0] + successes[1]) / (trials[0] + trials[1])
    # разница пропорций в датасетах
    difference = p1 - p2 
    # считаем статистику в ст.отклонениях стандартного нормального распределения
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/trials[0] + 1/trials[1]))
    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = st.norm(0, 1)
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    print('p-значение: ', p_value)
    if p_value < alpha:
        print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
    else:
        print(
            'Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными'
        )

In [ ]:
# напишем функцию, которая на вход получает порядковый номер группы в таблице и параметр альфа
# и возвращает результаты проверки гипотез
def exp_group (first, second, alpha):
    alpha = alpha
    total_cnt = ([logs_merged['total'].iloc[first], logs_merged['total'].iloc[second]])
    main_cnt = ([logs_merged['MainScreenAppear'].iloc[first], logs_merged['MainScreenAppear'].iloc[second]])
    offers_cnt = ([logs_merged['OffersScreenAppear'].iloc[first], logs_merged['OffersScreenAppear'].iloc[second]])
    cart_cnt = ([logs_merged['CartScreenAppear'].iloc[first], logs_merged['CartScreenAppear'].iloc[second]])
    payment_cnt = ([logs_merged['PaymentScreenSuccessful'].iloc[first], logs_merged['PaymentScreenSuccessful'].iloc[second]])
    
    print('Результаты теста по долям общего числа пользователей и с главного экрана')
    hypothesis_test(main_cnt, total_cnt, alpha)
    
    print('Результаты теста по долям общего числа пользователей и с экрана с каталогом')
    hypothesis_test(offers_cnt, total_cnt, alpha)
    
    print('Результаты теста по долям общего числа пользователей и с экрана с корзиной')
    hypothesis_test(cart_cnt, total_cnt, alpha)
    
    print('Результаты теста по долям общего числа пользователей и с экрана с оплатой')
    hypothesis_test(payment_cnt, total_cnt, alpha)

In [ ]:
# Сделаем проверку контрольных групп А/A
# выборки 246 и 247 в таблице logs_merged соответствуют индексам 0 и 1, alpha=0.05
exp_group (0, 1, 0.05)

**Вывод:**
Cтатистические критерии разницу между выборками 246 и 247 не находят. Т.к. контрольные группы оказались равны, мы можем быть уверены в точности проведенного тестирования.

### A/B-тест

Аналогично поступим с группой с изменённым шрифтом. Сравним результаты с каждой из контрольных групп в отдельности по каждому событию. Сравните результаты с объединённой контрольной группой. 

Для начала дополним таблицу `logs_merged` новой строкой с объединенной контрольной группой.


In [ ]:
logs_merged.loc['246_247'] = [
    logs_merged['MainScreenAppear'].iloc[0] + logs_merged['MainScreenAppear'].iloc[1],
    logs_merged['OffersScreenAppear'].iloc[0] + logs_merged['OffersScreenAppear'].iloc[1],
    logs_merged['CartScreenAppear'].iloc[0] + logs_merged['CartScreenAppear'].iloc[1],
    logs_merged['PaymentScreenSuccessful'].iloc[0] + logs_merged['PaymentScreenSuccessful'].iloc[1],
    logs_merged['total'].iloc[0] + logs_merged['total'].iloc[1]
]
logs_merged

In [ ]:
# Сделаем проверку контрольных групп А_246/B
# выборки 246 и 248 в таблице logs_merged соответствуют индексам 0 и 2, alpha=0.05
exp_group (0, 2, 0.05)

In [ ]:
# Сделаем проверку контрольных групп А_247/B
# выборки 247 и 248 в таблице logs_merged соответствуют индексам 1 и 2, alpha=0.05
exp_group (1, 2, 0.05)

In [ ]:
# Сделаем проверку контрольных групп А_246_247/B
# выборки 246_247 и 248 в таблице logs_merged соответствуют индексам 3 и 2, alpha=0.05
exp_group (3, 2, 0.05)

**Вывод по A/B-тесту:**

При проведении A/B-теста с двумя А-группами и объединенной А-группой выяснилось, что доля пользователей до изменения шрифтов и после не изменилась, поэтому изменение шрифта никак не влияет на количество пользователей приложения. 

При выполнении множественного тестирования возникает серьёзная проблема — быстрый рост вероятности допустить ошибку первого рода хотя бы в одном из тестов. В связи с этим мы должны скорректировать наши конечные p_value для сравнения их значений с alpha, но в этом нет необходимости, т.к. ни по одному из проведенных сравнений не обнаружилось статистически значимых различий.

**Общий вывод:**



Были предоставлены данные в одном файле, которые требовали предобработки: были изменены названия столбцов, изменен тип данных для столбца с датой и временем, а также добавлен столбец с датой, удалено 413 дубликатов. 

Всего в логе 7551 пользователь, но оказалось среди них много аномалий, а расчет 99 перцентиля показывает, что более 200 событий , совершенными одним пользователем считаются аномальными, поэтому от них избавились, таких пользователей оказалось 76. После удаления данных с аномальными пользователями, осталось 86% данных от исходного количества.

Хотя данные представлены за период с 25 июля по 7 августа 2019 года, но было принято решение отбросить все данные до 1 августа, т.к. они составляют менее 2% от общего числа и количество событий в эти даты очень мало, что может повлиять на объективность результатов в дальнейшем.

Таким образом было отброшено 37098 событий из 243713 и 93 пользователя из 7551, при этом в каждой из трёх экспериментальных групп осталось равномерное количество уникальных пользователей (2456, 2491, 2511), при этом каждый пользователь находится только в одной экспериментальной группе.

Всего в логе пять вариантов событий:

- показ главного экрана (49%),
- показ экрана с предложением (19%),
- показ экрана с корзиной (18%),
- оплата прошла успешно(18%),
- обучение (около 0%).

Событие, связанное с обучением, лишнее. Скорее всего оно не влияет на факт оплаты, т.к. видно на графике, что пользователи пользуются сайтом и без обучения.

98,5% пользователей посещали главный экран, 61% - переходили на страницу с предложением товаров, 49,6% - переходили в корзину и 47% пользователей успешно приобрели продукты в мобильном приложении.

Видно, что все события сужаются в воронку продаж, начиная с посещения приложения и заканчивая покупкой, но по пути пользователи отсеиваются и до покупки добираются только половина.

Пользователи могут попадать на разные экраны не последовательно, а также на каждом этапе пользователи отсеиваются, не доходя до оплаты.

Расчеты показали, что из главного экрана в каталог переходят только 62%, из них 81% переходит в корзину и из тех, кто перешел в корзину 95% пользователей оплачивают заказ. Таким образом, на втором событии "Экран предложений" теряется больше всего пользователей.

От первого события до оплаты доходят 45.5 % пользователей.

Cтатистические критерии разницу между выборками 246 и 247 не находят. Т.к. контрольные группы оказались равны, мы можем быть уверены в точности проведенного тестирования.

При проведении A/B-теста с двумя А-группами и объединенной А-группой выяснилось, что доля пользователей до изменения шрифтов и после не изменилась, поэтому **изменение шрифта никак не влияет на количество пользователей приложения.**

На основе проведенных исследований мои рекомендации заказчику следующие: т.к. больше всего пользователей теряется на этапе просмотра экрана, то возможно стоит сделать более яркой кнопку перехода в каталог или переместить ее на более видное место, также возможно следует собрать данные за более продолжительный период, т.к. мы отбросили целую неделю и осталась для исследований только одна неделя, а заказчик, возможно рассчитывал на период в две недели.